![](CRISP_DM.png)

In [1]:
#Importing required packages
import pandas as pd
import numpy as np
from sklearn import linear_model
from scipy import signal

#Importing packages for plotting
%matplotlib inline
pd.set_option('display.max_rows', 500)

# 4.1 Applying groupby on large relational data set

+ IMPORTANT: all written functions assume a data frame where the date is sorted!!

In [2]:
# creating a dataframe  using relational datafrom from the last notebook which contains all the data
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])

# Staring the data values in ascending order and resetting the  index
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


# 4.2 Test Data for  applying 'groupby'

In [3]:
# Generating a data frame for 'US' and 'Germany' after '2020-03-20'
test_data=pd_JH_data[((pd_JH_data['country']== 'US')|
                      (pd_JH_data['country']=='Germany'))&
                      ((pd_JH_data['date'] > '2020-05-20') & (pd_JH_data['date'] < '2020-05-29'))]

In [4]:
# Look at the data
test_data.head()

,date,state,country,confirmed
31963,2020-05-21,no,Germany,179021.0
32003,2020-05-21,no,US,1584512.0
32227,2020-05-22,no,Germany,179710.0
32267,2020-05-22,no,US,1608653.0
32494,2020-05-23,no,Germany,179986.0


In [5]:
# applying 'groupby' to check the maximum number of infected cases
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-05-28,no,182196.0
US,2020-05-28,no,1730260.0


### 4.2.1 Doubling time via regression_ calculation

In [6]:
# %load ../src/features/build_features.py

import numpy as np
import pandas as pd
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept = True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1,1)

    assert len(in_array) == 3
    reg.fit(X, y)
    intercept = reg.intercept_
    slope = reg.coef_

    return intercept/slope

In [7]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-05-28   182196.0
      US      2020-05-28  1730260.0

### 4.2.2 Rolling Regression

In [8]:
#Defining function for calculation of rolling regression
def rolling_reg(df_input,col='confirmed'):
    ''' Input has to be a data frame'''
    ''' Return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [9]:
# groupby data using 'state' & 'country' columns and then apply rolling regeression
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  31963           NaN
                32227           NaN
                32494    372.170639
                32760    582.550162
                33025    587.311618
                33293    414.470948
                33558    392.008658
                33825    364.738956
       US       32003           NaN
                32267           NaN
                32549     69.962594
                32801     76.467742
                33065     82.940492
                33333     88.200032
                33599     90.889959
                33865     83.166946
Name: confirmed, dtype: float64

In [10]:
# Applying the rollin egression to main DataFrame and resetting the index
pd_DR_result=pd_JH_data[['state','country','confirmed']]\
                        .groupby(['state','country'])\
                        .apply(rolling_reg,'confirmed').reset_index()

In [11]:
# Rename the columns for our convenience
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR','level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,998,NaN
4,Alberta,Canada,1231,NaN


In [12]:
#resetting the index
pd_JH_data=pd_JH_data.reset_index()

In [13]:
# merging the output of rolling regression with the main data frame 
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## 4.3 Filtering the data with groupby apply 


In [14]:
# define 'savgol_filter' for processing/filtering the data with the help of groupby function
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [15]:
# Applying groupby func on 'state' and 'country' column and after that applying 'savgol_filter'
pd_filtered_result=pd_JH_data[['state','country','confirmed']]\
                                .groupby(['state','country'])\
                                .apply(savgol_filter).reset_index()

In [16]:
#merging the results to the large dataframe using common column  name index
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


# 4.4 Filtered doubling rate

In [17]:
# filztering the doubling rate and storing it in a new DataFrame

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
56121,no,Zimbabwe,54928,59.167055
56122,no,Zimbabwe,55126,64.254047
56123,no,Zimbabwe,55423,54.757039
56124,no,Zimbabwe,55658,48.793130
56125,no,Zimbabwe,56125,51.934348


In [18]:
# merging as given above using column as index  
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
56121,56121,2020-08-19,no,Barbados,155.0,102.222222,154.6,127.833333
56122,56122,2020-08-19,no,Belarus,69801.0,657.430818,69806.0,609.726159
56123,56123,2020-08-19,no,Belgium,79479.0,167.132275,79379.2,208.222984
56124,56124,2020-08-19,no,Albania,7812.0,48.913738,7796.6,55.579100
56125,56125,2020-08-19,no,Zimbabwe,5643.0,32.495522,5563.4,51.934348


### Using Masks in Python

In [19]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [20]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
54839,54839,2020-08-15,no,Germany,224488.0,202.555505,224453.4,197.479778
55105,55105,2020-08-16,no,Germany,225007.0,369.126096,225621.2,192.168893
55370,55370,2020-08-17,no,Germany,226700.0,203.795961,226804.2,191.957008
55637,55637,2020-08-18,no,Germany,228120.0,145.588821,228159.1,178.778912
55902,55902,2020-08-19,no,Germany,229706.0,151.813262,229514.0,168.395527


In [21]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)